# Experimentation and uplift testing

In [1]:
# Load required packages
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('QVI_data.csv')
data

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream
...,...,...,...,...,...,...,...,...,...,...,...,...
264829,2370701,2018-12-08,88,240378,24,Grain Waves Sweet Chilli 210g,2,7.2,210,GRNWVES,YOUNG FAMILIES,Mainstream
264830,2370751,2018-10-01,88,240394,60,Kettle Tortilla ChpsFeta&Garlic 150g,2,9.2,150,KETTLE,YOUNG FAMILIES,Premium
264831,2370961,2018-10-24,88,240480,70,Tyrrells Crisps Lightly Salted 165g,2,8.4,165,TYRRELLS,OLDER FAMILIES,Budget
264832,2370961,2018-10-27,88,240481,65,Old El Paso Salsa Dip Chnky Tom Ht300g,2,10.2,300,OLD,OLDER FAMILIES,Budget


In [3]:
data['MONTH_ID'] = pd.to_datetime(data['DATE']).dt.strftime('%Y%m')
data['MONTH_ID'] = data['MONTH_ID'].astype(int)
data['MONTH_ID']

0         201810
1         201809
2         201903
3         201903
4         201811
           ...  
264829    201812
264830    201810
264831    201810
264832    201810
264833    201812
Name: MONTH_ID, Length: 264834, dtype: int32

---

In [5]:
tem = data.groupby('STORE_NBR')['MONTH_ID'].nunique() == 12
tem.value_counts()

MONTH_ID
True     260
False     12
Name: count, dtype: int64

In [6]:
tem2 = data[(data['STORE_NBR'].isin(tem[tem].index)) & (data['MONTH_ID'] < 201902)]
print(tem2.MONTH_ID.value_counts(), 'Number of unique store with full obs. is {}'.format(tem2.STORE_NBR.nunique()), sep='\n')

MONTH_ID
201812    22819
201807    22546
201808    22394
201810    22265
201901    22148
201811    21832
201809    21730
Name: count, dtype: int64
Number of unique store with full obs. is 260


In [45]:
metrics = tem2.groupby(['STORE_NBR','MONTH_ID'])[['TOT_SALES', 'LYLTY_CARD_NBR', 'TXN_ID', 'PROD_QTY']].agg({
    'TOT_SALES': 'sum', 'LYLTY_CARD_NBR' : 'nunique', 'TXN_ID' : 'nunique', 'PROD_QTY' : 'sum'
})

metrics['nTxnPerCust'] = metrics['TXN_ID'] / metrics['LYLTY_CARD_NBR']
metrics['nChipsPerTxn'] = metrics['PROD_QTY'] / metrics['LYLTY_CARD_NBR']
metrics['avgPricePerUnit'] = metrics['TOT_SALES'] / metrics['PROD_QTY']
metrics.rename(columns={'TOT_SALES': 'totSales', 'LYLTY_CARD_NBR': 'nCustomers'}, inplace=True)
metrics

totSales  nCustomers  TXN_ID  PROD_QTY  nTxnPerCust  \
STORE_NBR MONTH_ID                                                        
1         201807       206.9          49      52        62     1.061224   
          201808       176.1          42      43        54     1.023810   
          201809       278.8          59      62        75     1.050847   
          201810       188.1          44      45        58     1.022727   
          201811       192.6          46      47        57     1.021739   
...                      ...         ...     ...       ...          ...   
272       201809       304.7          32      36        71     1.125000   
          201810       430.6          44      50        99     1.136364   
          201811       376.2          41      45        87     1.097561   
          201812       403.9          47      47        89     1.000000   
          201901       423.0          46      50        96     1.086957   

                    nChipsPerTxn  avgPricePerUnit  
STORE_NBR MONTH_ID                                 
1         201807        1.265306         3.337097  
          201808        1.285714         3.261111  
          201809        1.271186         3.717333  
          201810        1.318182         3.243103  
          201811        1.239130         3.378947  
...                          ...              ...  
272       201809        2.218750         4.291549  
          201810        2.250000         4.349495  
          201811        2.121951         4.324138  
          201812        1.893617         4.538202  
          201901        2.086957         4.406250  

[1820 rows x 7 columns]

In [8]:
metrics.T.corr()

STORE_NBR                1                                            \
MONTH_ID              201807    201808    201809    201810    201811   
STORE_NBR MONTH_ID                                                     
1         201807    1.000000  0.999929  0.999155  0.999844  0.999956   
          201808    0.999929  1.000000  0.999107  0.999976  0.999937   
          201809    0.999155  0.999107  1.000000  0.999135  0.999352   
          201810    0.999844  0.999976  0.999135  1.000000  0.999865   
          201811    0.999956  0.999937  0.999352  0.999865  1.000000   
...                      ...       ...       ...       ...       ...   
272       201809    0.984462  0.985109  0.990137  0.985934  0.985237   
          201810    0.984354  0.984986  0.990072  0.985810  0.985127   
          201811    0.985337  0.985954  0.990880  0.986742  0.986110   
          201812    0.985436  0.986006  0.991162  0.986732  0.986302   
          201901    0.985179  0.985772  0.990821  0.986548  0.985972   

STORE_NBR                                    2                        ...  \
MONTH_ID              201812    201901    201807    201808    201809  ...   
STORE_NBR MONTH_ID                                                    ...   
1         201807    0.999851  0.999315  0.999665  0.998957  0.999211  ...   
          201808    0.999810  0.999210  0.999450  0.999035  0.998989  ...   
          201809    0.999359  0.999899  0.998019  0.999775  0.999666  ...   
          201810    0.999816  0.999153  0.999204  0.999182  0.998844  ...   
          201811    0.999777  0.999518  0.999536  0.999085  0.999405  ...   
...                      ...       ...       ...       ...       ...  ...   
272       201809    0.986596  0.988530  0.979784  0.991393  0.986988  ...   
          201810    0.986494  0.988448  0.979661  0.991312  0.986912  ...   
          201811    0.987370  0.989338  0.980799  0.992027  0.987864  ...   
          201812    0.987294  0.989745  0.981030  0.992011  0.988407  ...   
          201901    0.987190  0.989295  0.980646  0.991898  0.987852  ...   

STORE_NBR                271                           272            \
MONTH_ID              201811    201812    201901    201807    201808   
STORE_NBR MONTH_ID                                                     
1         201807    0.988003  0.989522  0.988035  0.985596  0.987070   
          201808    0.988801  0.990365  0.988861  0.986167  0.987574   
          201809    0.991959  0.992797  0.991694  0.991177  0.992359   
          201810    0.989673  0.991213  0.989756  0.986922  0.988269   
          201811    0.988464  0.989907  0.988418  0.986385  0.987816   
...                      ...       ...       ...       ...       ...   
272       201809    0.998769  0.997857  0.998211  0.999935  0.999811   
          201810    0.998724  0.997791  0.998157  0.999940  0.999812   
          201811    0.998770  0.997905  0.998195  0.999982  0.999908   
          201812    0.998066  0.997115  0.997342  0.999949  0.999911   
          201901    0.998573  0.997660  0.997955  0.999995  0.999922   

STORE_NBR                                                             
MONTH_ID              201809    201810    201811    201812    201901  
STORE_NBR MONTH_ID                                                    
1         201807    0.984462  0.984354  0.985337  0.985436  0.985179  
          201808    0.985109  0.984986  0.985954  0.986006  0.985772  
          201809    0.990137  0.990072  0.990880  0.991162  0.990821  
          201810    0.985934  0.985810  0.986742  0.986732  0.986548  
          201811    0.985237  0.985127  0.986110  0.986302  0.985972  
...                      ...       ...       ...       ...       ...  
272       201809    1.000000  0.999996  0.999980  0.999808  0.999963  
          201810    0.999996  1.000000  0.999979  0.999814  0.999967  
          201811    0.999980  0.999979  1.000000  0.999886  0.999992  
          201812    0.999808  0.999814  0.999886  1

In [66]:
tem = range(1,3)
lst = []
for i in tem:
    for j in range(i+1, max(tem)+2):
        lst.append([i,j])
lst

[[1, 2], [1, 3], [2, 3]]

In [39]:
metrics2 = metrics.reset_index()
metrics2['StoreMonth'] = metrics2['STORE_NBR'].astype(str) + '_' + metrics2['MONTH_ID'].astype(str)
metrics2.drop('MONTH_ID', axis=1, inplace=True)
metrics2

,STORE_NBR,totSales,nCustomers,TXN_ID,PROD_QTY,nTxnPerCust,nChipsPerTxn,avgPricePerUnit,StoreMonth
0,1,206.9,49,52,62,1.061224,1.265306,3.337097,1_201807
1,1,176.1,42,43,54,1.023810,1.285714,3.261111,1_201808
2,1,278.8,59,62,75,1.050847,1.271186,3.717333,1_201809
3,1,188.1,44,45,58,1.022727,1.318182,3.243103,1_201810
4,1,192.6,46,47,57,1.021739,1.239130,3.378947,1_201811
...,...,...,...,...,...,...,...,...,...
1815,272,304.7,32,36,71,1.125000,2.218750,4.291549,272_201809
1816,272,430.6,44,50,99,1.136364,2.250000,4.349495,272_201810
1817,272,376.2,41,45,87,1.097561,2.121951,4.324138,272_201811
1818,272,403.9,47,47,89,1.000000,1.893617,4.538202,272_201812


In [89]:
metrics2[metrics2['StoreMonth'] == '1_201809'].STORE_NBR.unique()

array([1], dtype=int64)

In [90]:
for i in metrics2['StoreMonth']:
    print(metrics2[metrics2['StoreMonth'] == i].STORE_NBR.unique())

[1]
[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[2]
[3]
[3]
[3]
[3]
[3]
[3]
[3]
[4]
[4]
[4]
[4]
[4]
[4]
[4]
[5]
[5]
[5]
[5]
[5]
[5]
[5]
[6]
[6]
[6]
[6]
[6]
[6]
[6]
[7]
[7]
[7]
[7]
[7]
[7]
[7]
[8]
[8]
[8]
[8]
[8]
[8]
[8]
[9]
[9]
[9]
[9]
[9]
[9]
[9]
[10]
[10]
[10]
[10]
[10]
[10]
[10]
[12]
[12]
[12]
[12]
[12]
[12]
[12]
[13]
[13]
[13]
[13]
[13]
[13]
[13]
[14]
[14]
[14]
[14]
[14]
[14]
[14]
[15]
[15]
[15]
[15]
[15]
[15]
[15]
[16]
[16]
[16]
[16]
[16]
[16]
[16]
[17]
[17]
[17]
[17]
[17]
[17]
[17]
[18]
[18]
[18]
[18]
[18]
[18]
[18]
[19]
[19]
[19]
[19]
[19]
[19]
[19]
[20]
[20]
[20]
[20]
[20]
[20]
[20]
[21]
[21]
[21]
[21]
[21]
[21]
[21]
[22]
[22]
[22]
[22]
[22]
[22]
[22]
[23]
[23]
[23]
[23]
[23]
[23]
[23]
[24]
[24]
[24]
[24]
[24]
[24]
[24]
[25]
[25]
[25]
[25]
[25]
[25]
[25]
[26]
[26]
[26]
[26]
[26]
[26]
[26]
[27]
[27]
[27]
[27]
[27]
[27]
[27]
[28]
[28]
[28]
[28]
[28]
[28]
[28]
[29]
[29]
[29]
[29]
[29]
[29]
[29]
[30]
[30]
[30]
[30]
[30]
[30]
[30]
[32]
[32]
[32]
[32]
[32]
[32]
[32]
[33]
[33]
[33

In [91]:
lst = []
for i in metrics2['StoreMonth']:
    for j in metrics2['StoreMonth']:
        if metrics2[metrics2['StoreMonth'] == i].STORE_NBR.unique() == metrics2[metrics2['StoreMonth'] == j].STORE_NBR.unique(): continue
        else: lst.append([metrics2[metrics2['StoreMonth'] == i], metrics2[metrics2['StoreMonth'] == j]])
lst

In [40]:
pairs = pd.merge(metrics2, metrics2, on='STORE_NBR', suffixes=('_1', '_2'))
pairs

,STORE_NBR,totSales_1,nCustomers_1,TXN_ID_1,PROD_QTY_1,nTxnPerCust_1,nChipsPerTxn_1,avgPricePerUnit_1,StoreMonth_1,totSales_2,nCustomers_2,TXN_ID_2,PROD_QTY_2,nTxnPerCust_2,nChipsPerTxn_2,avgPricePerUnit_2,StoreMonth_2
0,1,206.9,49,52,62,1.061224,1.265306,3.337097,1_201807,206.9,49,52,62,1.061224,1.265306,3.337097,1_201807
1,1,206.9,49,52,62,1.061224,1.265306,3.337097,1_201807,176.1,42,43,54,1.023810,1.285714,3.261111,1_201808
2,1,206.9,49,52,62,1.061224,1.265306,3.337097,1_201807,278.8,59,62,75,1.050847,1.271186,3.717333,1_201809
3,1,206.9,49,52,62,1.061224,1.265306,3.337097,1_201807,188.1,44,45,58,1.022727,1.318182,3.243103,1_201810
4,1,206.9,49,52,62,1.061224,1.265306,3.337097,1_201807,192.6,46,47,57,1.021739,1.239130,3.378947,1_201811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12735,272,423.0,46,50,96,1.086957,2.086957,4.406250,272_201901,304.7,32,36,71,1.125000,2.218750,4.291549,272_201809
12736,272,423.0,46,50,96,1.086957,2.086957,4.406250,272_201901,430.6,44,50,99,1.136364,2.250000,4.349495,272_201810
12737,272,423.0,46,50,96,1.086957,2.086957,4.406250,272_201901,376.2,41,45,87,1.097561,2.121951,4.324138,272_201811
12738,272,423.0,46,50,96,1.086957,2.086957,4.406250,272_201901,403.9,47,47,89,1.000000,1.893617,4.538202,272_201812


In [46]:
diff = pd.DataFrame({
    'Diff_totSales' : pairs['totSales_1'] - pairs['totSales_2'],
    'Diff_nCustomers' : pairs['nCustomers_1'] - pairs['nCustomers_2'],
    'Diff_nTxnPerCust' : pairs['nTxnPerCust_1'] - pairs['nTxnPerCust_2'],
    'Diff_nChipsPerTxn' : pairs['nChipsPerTxn_1'] - pairs['nChipsPerTxn_2'],
    'Diff_avgPricePerUnit' : pairs['avgPricePerUnit_1'] - pairs['avgPricePerUnit_2'],
    'StoreMonth_1' : pairs['StoreMonth_1'],
    'StoreMonth_2' : pairs['StoreMonth_2']
})
# remove pairs with same store of every month
diff = diff[diff['StoreMonth_1'] != diff['StoreMonth_2']]
diff

,Diff_totSales,Diff_nCustomers,Diff_nTxnPerCust,Diff_nChipsPerTxn,Diff_avgPricePerUnit,StoreMonth_1,StoreMonth_2
1,30.80,7,0.037415,-0.020408,0.075986,1_201807,1_201808
2,-71.90,-10,0.010377,-0.005880,-0.380237,1_201807,1_201809
3,18.80,5,0.038497,-0.052876,0.093993,1_201807,1_201810
4,14.30,3,0.039485,0.026176,-0.041851,1_201807,1_201811
5,17.30,7,-0.057823,-0.091837,0.010781,1_201807,1_201812
...,...,...,...,...,...,...,...
12734,50.15,2,-0.003953,0.155138,0.019779,272_201901,272_201808
12735,118.30,14,-0.038043,-0.131793,0.114701,272_201901,272_201809
12736,-7.60,2,-0.049407,-0.163043,0.056755,272_201901,272_201810
12737,46.80,5,-0.010604,-0.034995,0.082112,272_201901,272_201811
